In [1]:
# Import python directory
import sys
sys.path.append('..')

import torch
from PIL import Image
import matplotlib.pyplot as plt
from src.domain import Domain
from src.interpolation import SplineInter
from src.transformations import Affine2d
from src.plotting import view_image_2d, plot_grid_2d
from src.super_resolution_tools import *
from src.transformations import Affine2d
from src.LinearOperator import LinearOperator
from src.lanczos import lanczos_tridiag
# from src.conjugate_gradient import conjugate_gradient
from src.utils import * 
from src.optimization import *
import torch.func as func
from src.distance import SSDDistance
import time

torch.set_default_dtype(torch.float64)

In [2]:
As = torch.randn(10, 5,5)
As = As @ As.transpose(1,2)
As = As + 0.1 * torch.eye(5)
def A(x):
    # multiply A with x along 0th dimension without using einsum
    return torch.sum(As * (x**2).unsqueeze(1).unsqueeze(1), dim=0)


x = torch.randn(10)
Ax = A(x)
b = torch.sum(Ax,dim=1)
print(Ax)
print(b)



tensor([[44.5249,  1.3237,  3.7101,  6.1252,  5.2042],
        [ 1.3237, 52.4561, 14.6625,  5.8509, -8.5327],
        [ 3.7101, 14.6625, 52.4407,  1.6485,  2.7136],
        [ 6.1252,  5.8509,  1.6485, 43.3197, -2.9519],
        [ 5.2042, -8.5327,  2.7136, -2.9519, 40.1944]])
tensor([60.8881, 65.7605, 75.1754, 53.9924, 36.6276])


In [3]:
def conjugate_gradient(A, b, x0, max_iter=50, tol=1e-15):
    x = x0.clone()
    r = b - A@x
    p = r.clone()
    rsold = r.dot(r)
        
    for i in range(max_iter):
        Ap = A@p
        alpha = rsold / p.dot(Ap)
        x = x + alpha * p  
        r = r - alpha * Ap  
        rsnew = r.dot(r)
        if torch.sqrt(rsnew/rsold) < tol:
            print(f"tol reached {i}")
            break
        p = r + (rsnew / rsold) * p
        rsold = rsnew
    # print(f"residual norm: {torch.sqrt(rsnew/r0)}")
    return x
    


def res_fn(x, cg_iter):
    Ax = A(x)
    n = Ax.shape[1]
    f0 = conjugate_gradient(Ax, b, torch.zeros(n), max_iter=cg_iter, tol=1e-15)
    
    return f0

def Jac_fn(wp_vec, cg_iter):
    return func.jacfwd(res_fn)(wp_vec, cg_iter).detach().squeeze()



In [4]:
cg_iter = 4
x0 = torch.randn(10)
res = res_fn(x0, cg_iter)
J = Jac_fn(x0, cg_iter)
dw = torch.randn_like(x0)  
dres = J @ dw

for i in range(40):
    h = 0.5**i
    rest = res_fn(x0 + h * dw, cg_iter)
    
    err0 = torch.norm(res - rest)
    err1 = torch.norm(res + h * dres - rest)
    print(f"Error at h={h:.3e}: {err0:.3e} vs {err1:.3e}")
    


Error at h=1.000e+00: 1.264e+00 vs 1.127e+00
Error at h=5.000e-01: 7.744e-01 vs 3.899e-01
Error at h=2.500e-01: 4.361e-01 vs 1.143e-01
Error at h=1.250e-01: 2.326e-01 vs 3.074e-02
Error at h=6.250e-02: 1.208e-01 vs 8.037e-03
Error at h=3.125e-02: 6.122e-02 vs 2.051e-03
Error at h=1.562e-02: 3.088e-02 vs 5.164e-04
Error at h=7.812e-03: 1.551e-02 vs 1.295e-04
Error at h=3.906e-03: 7.770e-03 vs 3.243e-05
Error at h=1.953e-03: 3.889e-03 vs 8.114e-06
Error at h=9.766e-04: 1.946e-03 vs 2.029e-06
Error at h=4.883e-04: 9.731e-04 vs 5.074e-07
Error at h=2.441e-04: 4.866e-04 vs 1.269e-07
Error at h=1.221e-04: 2.433e-04 vs 3.172e-08
Error at h=6.104e-05: 1.217e-04 vs 7.930e-09
Error at h=3.052e-05: 6.083e-05 vs 1.982e-09
Error at h=1.526e-05: 3.042e-05 vs 4.956e-10
Error at h=7.629e-06: 1.521e-05 vs 1.239e-10
Error at h=3.815e-06: 7.604e-06 vs 3.098e-11
Error at h=1.907e-06: 3.802e-06 vs 7.744e-12
Error at h=9.537e-07: 1.901e-06 vs 1.935e-12
Error at h=4.768e-07: 9.506e-07 vs 4.837e-13
Error at h